# Set up assets that are needed for data preprocessing

References:

https://github.com/Cledge-org/cledge/blob/dev/features/college_search_tool/assets/

https://github.com/Cledge-org/cledge/blob/dev/features/college_search_tool/src/college_search_index_builder.ipynb

## 1. Update necessary fields

In [60]:
import pandas as pd
import json
from pyspark import SparkContext
from pyspark.sql import *
import pyspark.sql.functions as F

In [62]:
# Initialize SparkContext
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [53]:
# read the data
data = pd.read_csv("./data/Most-Recent-Cohorts-All-Data-Elements.csv")
data_dict = pd.read_csv("./data/Institution_data_dictionary.csv")

data.shape

/Users/minzhou/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3331: DtypeWarning: Columns (6,9,1351,1352,1353,1354,1355,1356,1357,1358,1359,1360,1361,1362,1363,1364,1365,1379,1393,1394,1395,1396,1397,1398,1399,1400,1401,1402,1403,1404,1405,1406,1407,1408,1409,1410,1412,1413,1427,1428,1431,1432,1503,1517,1532,1533,1534,1535,1536,1537,1538,1539,1540,1542,1543,1544,1545,1546,1547,1548,1549,1550,1551,1552,1553,1554,1555,1556,1557,1558,1559,1560,1561,1567,1568,1573,1574,1575,1576,1577,1581,1582,1587,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599,1600,1601,1602,1604,1605,1606,1608,1610,1611,1614,1615,1616,1619,1620,1621,1622,1623,1624,1625,1626,1627,1628,1629,1636,1638,1640,1643,1644,1648,1653,1654,1655,1656,1657,1658,1659,1660,1661,1662,1664,1666,1669,1670,1674,1679,1680,1681,1682,1683,1684,1685,1686,1687,1688,1689,1690,1691,1692,1693,1695,1697,1700,1701,1702,1703,1704,1705,1706,1707,1711,1725,1726,1727,1728,1729,1743,1815,1816,1817,1818,1823,1824,18

(6694, 2989)

In [54]:
# read previous fields.json file
with open("./assets/fields.json") as prevFields:
    prevfields_json = json.load(prevFields)

# Select columns
data_dict_name_map = data_dict[['VARIABLE NAME', 'developer-friendly name']]

# re-insert all variable-name & developer-friendly name pairs
for variable_name in prevfields_json.keys():
    prevfields_json[variable_name] = data_dict_name_map.loc[data_dict_name_map["VARIABLE NAME"] == variable_name, "developer-friendly name"].values[0]

# write the updated fields into fields.json
with open("./assets/fields.json", "w") as curFields:
    curFields.write(json.dumps(prevfields_json, indent=4))

# filter dataset with only fields in the fields.json
with open("./assets/fields.json") as curFields:
    dataFields = json.load(curFields)
    data = data[list(dataFields.keys())]
data.shape

(6694, 472)

## 2. Update and Check Data Types

In [44]:
# read previous datatypes.json file
with open("./assets/datatypes.json") as prevDTypes:
    prevDTypes_json = json.load(prevDTypes)

# Select API data type and VARIABLE NAME from data_dict
data_dict_data_type = data_dict[['VARIABLE NAME', 'API data type']]

for dtype_name in prevDTypes_json.keys():
    prevDTypes_json[dtype_name] = data_dict_data_type.loc[data_dict_data_type['VARIABLE NAME'] == dtype_name, "API data type"].values[0]

# write the updated fields into datatypes.json
with open("./assets/datatypes.json", "w") as curDTypes:
    curDTypes.write(json.dumps(prevDTypes_json, indent=4))

# manually edit "autocomplete" data type into "string" ("INSTNM" and "CITY")

In [61]:
# save current data into csv
data.to_csv("./data/Necessary-Fields-Data.csv")

In [71]:
# read dataset using PySpark
df = spark.read.csv("./data/Necessary-Fields-Data.csv", header=True, inferSchema=True)

In [65]:
# use PySpark to check and convert data types
with open('./assets/datatypes.json') as f:
    datatypes = json.load(f)

# cast columns to correct datatypes
for field, datatype in datatypes.items(): # takes a few mins to run
    if field not in df.columns:
        continue
    curr_type = dict(df.dtypes)[field]
    if curr_type != datatype and not datatype.startswith(curr_type):
        df = df.withColumn(field, F.col(field).cast(datatype))

In [72]:
print(len(df.columns))

473


## 3. Filter Empty (Null) Values

In [80]:
# Replace all "NULL" values in dataframe with literal null values so that isnull() can be used
df = df.replace({'NULL': None, 'null': None})

In [84]:
null_counts = df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in df.columns]).collect()[0].asDict()

In [85]:
# Drop fields that are completely null
num_rows = df.count()

for field, null_count in null_counts.items():
    if null_count >= num_rows: # all values of field are null, drop the field
        df = df.drop(field)
print(len(df.columns)) # number of fields that are not completely null

447


In [86]:
# Save dictionary of null value counts for reference
with open('./assets/null_counts.json', 'w') as f:
    json.dump(null_counts, f)

In [87]:
# Test that casting worked: Howard University should be true for HBCU
df.where(df.INSTNM.contains('Howard')).select(["INSTNM", "HBCU"]).show(truncate=False)

+---------------------------------+----+
|INSTNM                           |HBCU|
+---------------------------------+----+
|Howard University                |1.0 |
|Howard Community College         |0.0 |
|Specs Howard School of Media Arts|0.0 |
|Howard College                   |0.0 |
|Howard Payne University          |0.0 |
+---------------------------------+----+



In [88]:
# replace "CCBASIC" with any field name (or add other field names to the select list) from fields.json to print out specific fields for UW campuses
df.where(df.INSTNM.contains('University of Washington')).select(["INSTNM", "CCBASIC"]).show(truncate=False)

+---------------------------------------+-------+
|INSTNM                                 |CCBASIC|
+---------------------------------------+-------+
|University of Washington-Seattle Campus|15.0   |
|University of Washington-Bothell Campus|18.0   |
|University of Washington-Tacoma Campus |18.0   |
+---------------------------------------+-------+



In [ ]:
# save the dataframe
df_filename = './data/college-search-data.parquet'
df.write.save(df_filename)

In [ ]:
sc.stop()

In [59]:
# with open("./assets/datatypes.json") as curDTypes:
#     cur_dataTypes = json.load(curDTypes)

# for column in data.columns:
#     col_dataType = str(data[column].dtypes)
#     expect_dataType = cur_dataTypes[column]
#     is_equal = False
#     if expect_dataType == "integer":
#         if not col_dataType == "int64":
#             data[column].astype("int", errors='ignore')
#     elif expect_dataType == "float":
#         if not col_dataType == "float64":
#             data[column].astype("float", errors='ignore')
#     else:
#         if not col_dataType == "object":
#             data[column].astype("string", errors='ignore')

In [58]:
# for column in data.columns:
#     col_dataType = str(data[column].dtypes)
#     expect_dataType = cur_dataTypes[column]
#     is_equal = False
#     if expect_dataType == "integer":
#         if col_dataType == "int64":
#             is_equal = True
#     elif expect_dataType == "float":
#         if col_dataType == "float64":
#             is_equal = True
#     else:
#         if col_dataType == "string" or col_dataType == "object":
#             is_equal = True
#     if not is_equal:
#         print(column)